# Recommendation System using Sentence Transformer


## Load csv into Pandas Dataframe


In [1]:
import pandas as pd
sent_df = pd.read_csv("../netflix_titles.csv")


In [2]:
# This is a sentence-transformers model: It maps sentences & paragraphs to a 768 dimensional dense vector space and can be used for tasks like clustering or semantic search.

from sentence_transformers import SentenceTransformer

model = SentenceTransformer("paraphrase-distilroberta-base-v1")


## Find Embeddings for all show descriptions in dataset.


In [3]:
descriptions = sent_df["description"].tolist()
# print(descriptions)
des_embeddings = []
for i, des in enumerate(descriptions):
    des_embeddings.append(model.encode(des))


## For a query show_id let's find the top ten shows with the highest cosine similarity.


In [4]:
import torch
from sentence_transformers import util


def recommend(query):
    query_embedded = model.encode(query)
    cosine_scores = util.pytorch_cos_sim(query_embedded, des_embeddings)
    top10_matches = torch.argsort(cosine_scores, dim=-1, descending=True).tolist()[0][
        1:11
    ]
    return top10_matches


## Movie Recommender using sentence transformers


In [5]:
title = "The Conjuring"
query_show_des = sent_df.loc[sent_df["title"] == title]["description"].to_list()[0]

recommended_results = recommend(query_show_des)
recommended_results = [x + 1 for x in recommended_results]

for i in range(len(recommended_results)):
    print(
        sent_df["title"].loc[
            sent_df["show_id"] == str("s" + str(recommended_results[i]))
        ]
    )


7260    Laddaland
Name: title, dtype: object
8279    The Diabolical
Name: title, dtype: object
1637    Don’t Listen
Name: title, dtype: object
7361    Lupt
Name: title, dtype: object
7936    Sardaar ji
Name: title, dtype: object
836    Ghost Lab
Name: title, dtype: object
5332    Demonic
Name: title, dtype: object
6845    Ghost House
Name: title, dtype: object
8060    Soul to Keep
Name: title, dtype: object
7488    Monster Family
Name: title, dtype: object


/home/abishek-as/Netflix-NLP/Netflix-venv/lib/python3.8/site-packages/sentence_transformers/util.py:32: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  b = torch.tensor(b)
